In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')
%run '../functions.py'
%run '../classes.py'

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/adammajczyk/nltk_data..

In [2]:
save_path = 'grid_search_results_new.json'
grids_path = '../grid_search_grids.json'

In [3]:
model_w2v_settings = return_best_model()

In [4]:
model_w2v = Word2VecModel(model_w2v_settings)

In [5]:
# read preprocessed data from pickle file
df = pd.read_pickle('data/preprocessed_titles_labels.pkl')
df.head()

,title,is_clickbait,text,dataset,sample
0,"[seven, essay, read, sex, scene, actually, sex...",1,NaN,clickbait-dataset,train
1,"[horrible, truth, bagel]",1,NaN,clickbait-dataset,train
2,"[entire, demlib, party, infested, rat, much, c...",1,"— Susan? (@GaetaSusan) October 27, 2016 WikiLe...",fake-news,train
3,"[beginning, new, life, america, mentor, side, ...",0,"OAKLAND, Calif. — Pascal Serugendo was only...",fake-news,train
4,"[imam, pleads, guilty, new, york, subway, bomb...",0,NaN,clickbait-dataset,train


In [6]:
df['sample'].value_counts()


sample
train    46618
test      2590
val2      1296
val1      1295
Name: count, dtype: int64

In [7]:
# split data into train and test stratified by y
from sklearn.preprocessing import StandardScaler
import os
import pickle

df['title_vector'] = [get_word_vectors(model_w2v, title, aggregation='mean') for title in df['title']]
df

,title,is_clickbait,text,dataset,sample,title_vector
0,"[seven, essay, read, sex, scene, actually, sex...",1,NaN,clickbait-dataset,train,"[-0.40359393, 0.58101, 0.68450487, -0.9439005,..."
1,"[horrible, truth, bagel]",1,NaN,clickbait-dataset,train,"[0.46905693, 0.5450296, -0.19201148, 0.4663539..."
2,"[entire, demlib, party, infested, rat, much, c...",1,"— Susan? (@GaetaSusan) October 27, 2016 WikiLe...",fake-news,train,"[0.3386298, -0.66440797, 0.594211, 0.2816573, ..."
3,"[beginning, new, life, america, mentor, side, ...",0,"OAKLAND, Calif. — Pascal Serugendo was only...",fake-news,train,"[-0.57141453, 1.092386, 1.7740865, 0.8420436, ..."
4,"[imam, pleads, guilty, new, york, subway, bomb...",0,NaN,clickbait-dataset,train,"[-0.023485124, 0.76326317, 0.21076646, 2.82912..."
...,...,...,...,...,...,...
51798,"[tamil, rebel, lose, ground, sri, lanka, crisi...",0,NaN,clickbait-dataset,val2,"[-2.9222271, 1.81281, 1.2188933, 1.4079493, -0..."
51799,"[lot, people, furious, magazine, shaming, woma...",1,NaN,clickbait-dataset,val2,"[0.62786365, 0.07262453, 1.4181519, -0.5978753..."
51800,"[eleven, reason, bindi, irwin, incredible, rol...",1,NaN,clickbait-dataset,val2,"[1.0960542, 0.9256813, 0.9718578, -1.7881821, ..."
51801,"[donald, trump, backslides, campaign, promise,...",0,President Donald Trump has slid back from his ...,fake-news,val2,"[-2.3169072, 0.24867076, 0.009470575, 2.800947..."


In [8]:
df['title_vector'][0]

array([-0.40359393,  0.58101   ,  0.68450487, -0.9439005 ,  1.0436215 ,
        1.4960831 , -1.5474931 , -3.395665  ,  2.0742133 ,  0.6147578 ,
        0.06466653, -0.2790067 ,  1.0890667 , -0.5697715 ,  2.480676  ,
       -0.36809966, -1.7050891 ,  0.20265865, -1.14607   ,  0.47789478,
       -1.8076416 ,  0.8146902 , -0.62458795, -0.77249545, -0.47756815,
        0.51792186,  0.3093162 ,  0.8395704 , -0.24838673,  0.5574209 ,
       -0.24416532,  0.13703738, -0.9198161 ,  0.6077911 ,  0.37606242,
        1.6745015 , -1.2676517 , -1.0577408 , -0.06308307,  0.47314626,
       -1.9368892 ,  1.24171   ,  0.53468794, -1.3008615 , -1.5342575 ,
       -0.45216206,  0.09969594,  0.79413736, -0.5304332 ,  1.0210458 ],
      dtype=float32)

In [9]:
variables_to_drop = get_dimensions_to_drop()
variables_to_drop

[32,
 31,
 46,
 27,
 2,
 17,
 20,
 10,
 24,
 34,
 42,
 39,
 30,
 6,
 33,
 48,
 0,
 22,
 9,
 40,
 41,
 7,
 11]

In [10]:
df['title_vector'] = [drop_dimensions_from_vector(vector, variables_to_drop) for vector in df['title_vector']]

In [11]:
df['title_vector'][0]

array([ 0.58101   , -0.9439005 ,  1.0436215 ,  1.4960831 ,  2.0742133 ,
        1.0890667 , -0.5697715 ,  2.480676  , -0.36809966, -1.7050891 ,
       -1.14607   ,  0.47789478,  0.8146902 , -0.77249545,  0.51792186,
        0.3093162 , -0.24838673,  0.5574209 ,  1.6745015 , -1.2676517 ,
       -1.0577408 , -0.06308307, -1.3008615 , -1.5342575 , -0.45216206,
        0.79413736,  1.0210458 ], dtype=float32)

In [12]:
X_train = np.vstack(df[df['sample']=='train']['title_vector'])
X_test = np.vstack(df[df['sample']=='val2']['title_vector'])

y_train = df[df['sample']=='train']['is_clickbait']
y_test = df[df['sample']=='val2']['is_clickbait']

# scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
import json

with open(grids_path, 'r') as f:
    model_settings = json.load(f)

model_settings
grids = model_settings['grid_search_grids']
grids

{'decision_tree_grid': {'max_depth': [None, 3, 4, 5, 6, 7, 8],
  'max_features': ['sqrt', 'log2', None]},
 'catboost_grid': {'iterations': [100, 250, 500],
  'learning_rate': [None, 0.001, 0.01, 0.1, 0.2, 0.3],
  'depth': [3, 4, 5, 6, 7, 8],
  'l2_leaf_reg': [1, 3, 5, 7, 9]},
 'lightgbm_grid': {'n_estimators': [50, 100, 250, 500],
  'learning_rate': [None, 0.01, 0.1, 0.2, 0.3],
  'max_depth': [-1, 3, 4, 5, 6, 7, 8],
  'num_leaves': [15, 31, 62, 93, 124, 155, 187, 217]},
 'xgboost_grid': {'n_estimators': [10, 25, 50, 100, 250],
  'learning_rate': [None, 0.001, 0.01, 0.1, 0.2, 0.3],
  'max_depth': [None, 3, 4, 5, 6, 7, 8],
  'gamma': [0, 0.01, 0.1, 0.5, 1, 5]},
 'random_forest_grid': {'n_estimators': [50, 100, 250, 500],
  'max_depth': [None, 3, 4, 5, 6, 7, 8],
  'max_features': ['sqrt', 'log2', None]}}

In [14]:
models_to_CV = list()

for model_name, grid in grids.items():
    # check if model has already been trained - if 'best_params' exists in grid
    if 'best_params' in grid.keys():
        print(f'Model {model_name} already trained')
        continue
    models_to_CV.append((model_name))
models_to_CV

# remove _grid from model names
models_to_CV = [model_name.replace('_grid', '') for model_name in models_to_CV]
models_to_CV

['decision_tree', 'catboost', 'lightgbm', 'xgboost', 'random_forest']

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [16]:
def return_model_with_param(model_name, param = dict()):
    if model_name == 'decision_tree':
        model = DecisionTreeClassifier(**param)
    elif model_name == 'random_forest':
        model = RandomForestClassifier(**param)
    elif model_name == 'xgboost':
        model = XGBClassifier(**param)
    elif model_name == 'lightgbm':
        model = LGBMClassifier(**param, verbose=-1)
    elif model_name == 'catboost':
        model = CatBoostClassifier(**param,verbose=0)
    return model

In [17]:
models_to_CV = [
    'xgboost',
    'lightgbm', 
    'decision_tree', 
    'random_forest',  
    'catboost'
]

In [18]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score
from tqdm.notebook import tqdm

# read existing results
try:
    with open(save_path, 'r') as f:
        grid_search_results = json.load(f)
    print('Loaded existing results')
except:
    grid_search_results = dict()
    print('No existing results found - creating new dict')
for model_name in models_to_CV:
    grid_search_results[model_name] = dict()

for model_name in tqdm(models_to_CV, desc = 'Models'):

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # get grid
    grid = grids[model_name+'_grid']

    # generate all combinations of parameters
    import itertools

    keys, values = zip(*grid.items())
    combinations_dicts = [dict(zip(keys, v)) for v in itertools.product(*values)]

    # do grid search
    
    for params in combinations_dicts:
        grid_search_results[model_name][str(params)] = dict()

    for params in tqdm(combinations_dicts, desc = 'Grid combinations search for model {}'.format(model_name)):
       
        model = return_model_with_param(model_name, params)
        scores_auc_cv_val = list()
        scores_auc_val2 = list()
        scores_auc_train = list()

        scores_f1_cv_val = list()
        scores_f1_val2 = list()
        scores_f1_train = list()

        for train_index, val_index in kf.split(X_train_scaled, y_train):
            X_train_kf = X_train_scaled[train_index]
            X_val_kf = X_train_scaled[val_index]
            y_train_kf = y_train.iloc[train_index]
            y_val_kf = y_train.iloc[val_index]

            model.fit(X_train_kf, y_train_kf)

            # print(model)

        
            scores_auc_cv_val.append(roc_auc_score(y_val_kf, model.predict_proba(X_val_kf)[:,1]))
            scores_auc_val2.append(roc_auc_score(y_test, model.predict_proba(X_test_scaled)[:,1]))
            scores_auc_train.append(roc_auc_score(y_train_kf, model.predict_proba(X_train_kf)[:,1]))

            y_pred_cv_val = model.predict_proba(X_val_kf)[:,1]
            y_pred_val2 = model.predict_proba(X_test_scaled)[:,1]
            y_pred_train = model.predict_proba(X_train_kf)[:,1]

            # print(y_pred_cv_val[y_pred_cv_val>0.5])

            y_pred_cv_val = np.where(y_pred_cv_val > 0.5, 1, 0)
            y_pred_val2 = np.where(y_pred_val2 > 0.5, 1, 0)
            y_pred_train = np.where(y_pred_train > 0.5, 1, 0)


            
            scores_f1_cv_val.append(f1_score(y_val_kf, model.predict(X_val_kf)))
            scores_f1_val2.append(f1_score(y_test, model.predict(X_test_scaled)))
            scores_f1_train.append(f1_score(y_train_kf, model.predict(X_train_kf)))
            break

        # print(scores_f1_train, scores_f1_cv_val, scores_f1_val2)
        # print(scores_auc_train, scores_auc_cv_val, scores_auc_val2)

        grid_search_results[model_name][str(params)]['mean_train_f1'] = np.mean(scores_f1_train)
        grid_search_results[model_name][str(params)]['mean_cv_val_f1'] = np.mean(scores_f1_cv_val)
        grid_search_results[model_name][str(params)]['mean_val2_f1'] = np.mean(scores_f1_val2)

        grid_search_results[model_name][str(params)]['mean_train_auc'] = np.mean(scores_auc_train)
        grid_search_results[model_name][str(params)]['mean_cv_val_auc'] = np.mean(scores_auc_cv_val)
        grid_search_results[model_name][str(params)]['mean_val2_auc'] = np.mean(scores_auc_val2)

        grid_search_results[model_name][str(params)]['f1_diff_cv_val'] = np.mean(scores_f1_train) - np.mean(scores_f1_cv_val)
        grid_search_results[model_name][str(params)]['auc_diff_cv_val'] = np.mean(scores_auc_train) - np.mean(scores_auc_cv_val)

        grid_search_results[model_name][str(params)]['f1_diff_val2'] = np.mean(scores_f1_train) - np.mean(scores_f1_val2)
        grid_search_results[model_name][str(params)]['auc_diff_val2'] = np.mean(scores_auc_train) - np.mean(scores_auc_val2)

        grid_search_results[model_name][str(params)]['params'] = params

        # save results to json file
        with open(save_path, 'w') as f:
            json.dump(grid_search_results, f)
    



    print(grid_search_results[model_name])
    
# grid_search_results

No existing results found - creating new dict


Models:   0%|          | 0/5 [00:00<?, ?it/s]

Grid combinations search for model xgboost:   0%|          | 0/1260 [00:00<?, ?it/s]

{"{'n_estimators': 10, 'learning_rate': None, 'max_depth': None, 'gamma': 0}": {'mean_train_f1': 0.8610807389077516, 'mean_cv_val_f1': 0.8391384051329056, 'mean_val2_f1': 0.8497495826377295, 'mean_train_auc': 0.9418914385388312, 'mean_cv_val_auc': 0.9256603230821099, 'mean_val2_auc': 0.9272870211549458, 'f1_diff_cv_val': 0.021942333774846, 'auc_diff_cv_val': 0.01623111545672129, 'f1_diff_val2': 0.011331156270022125, 'auc_diff_val2': 0.014604417383885404, 'params': {'n_estimators': 10, 'learning_rate': None, 'max_depth': None, 'gamma': 0}}, "{'n_estimators': 10, 'learning_rate': None, 'max_depth': None, 'gamma': 0.01}": {'mean_train_f1': 0.8610807389077516, 'mean_cv_val_f1': 0.8391384051329056, 'mean_val2_f1': 0.8497495826377295, 'mean_train_auc': 0.9418914385388312, 'mean_cv_val_auc': 0.9256603230821099, 'mean_val2_auc': 0.9272870211549458, 'f1_diff_cv_val': 0.021942333774846, 'auc_diff_cv_val': 0.01623111545672129, 'f1_diff_val2': 0.011331156270022125, 'auc_diff_val2': 0.0146044173838

Grid combinations search for model lightgbm:   0%|          | 0/1120 [00:00<?, ?it/s]

...

{"{'max_depth': None, 'max_features': 'sqrt'}": {'mean_train_f1': 1.0, 'mean_cv_val_f1': 0.7748714019919012, 'mean_val2_f1': 0.794162826420891, 'mean_train_auc': 1.0, 'mean_cv_val_auc': 0.7792836455647264, 'mean_val2_auc': 0.7935677530017152, 'f1_diff_cv_val': 0.2251285980080988, 'auc_diff_cv_val': 0.2207163544352736, 'f1_diff_val2': 0.20583717357910902, 'auc_diff_val2': 0.20643224699828477, 'params': {'max_depth': None, 'max_features': 'sqrt'}}, "{'max_depth': None, 'max_features': 'log2'}": {'mean_train_f1': 1.0, 'mean_cv_val_f1': 0.7757141305528402, 'mean_val2_f1': 0.7830853563038371, 'mean_train_auc': 1.0, 'mean_cv_val_auc': 0.7785654961009911, 'mean_val2_auc': 0.7862635791881075, 'f1_diff_cv_val': 0.22428586944715978, 'auc_diff_cv_val': 0.22143450389900887, 'f1_diff_val2': 0.21691464369616287, 'auc_diff_val2': 0.21373642081189248, 'params': {'max_depth': None, 'max_features': 'log2'}}, "{'max_depth': None, 'max_features': None}": {'mean_train_f1': 1.0, 'mean_cv_val_f1': 0.78389553

Grid combinations search for model random_forest:   0%|          | 0/84 [00:00<?, ?it/s]

{"{'n_estimators': 50, 'max_depth': None, 'max_features': 'sqrt'}": {'mean_train_f1': 0.9998633767795174, 'mean_cv_val_f1': 0.8572733500799269, 'mean_val2_f1': 0.8712871287128713, 'mean_train_auc': 0.9999999741074402, 'mean_cv_val_auc': 0.9397994631023846, 'mean_val2_auc': 0.9398787402325137, 'f1_diff_cv_val': 0.14259002669959053, 'auc_diff_cv_val': 0.060200511005055635, 'f1_diff_val2': 0.12857624806664614, 'auc_diff_val2': 0.060121233874926516, 'params': {'n_estimators': 50, 'max_depth': None, 'max_features': 'sqrt'}}, "{'n_estimators': 50, 'max_depth': None, 'max_features': 'log2'}": {'mean_train_f1': 0.999836038694868, 'mean_cv_val_f1': 0.8598515134209024, 'mean_val2_f1': 0.8637873754152824, 'mean_train_auc': 0.9999999151299432, 'mean_cv_val_auc': 0.9391355557166479, 'mean_val2_auc': 0.9407185058128454, 'f1_diff_cv_val': 0.13998452527396565, 'auc_diff_cv_val': 0.06086435941329538, 'f1_diff_val2': 0.13604866327958565, 'auc_diff_val2': 0.0592814093170978, 'params': {'n_estimators': 50

Grid combinations search for model catboost:   0%|          | 0/540 [00:00<?, ?it/s]

{"{'iterations': 100, 'learning_rate': None, 'depth': 3, 'l2_leaf_reg': 1}": {'mean_train_f1': 0.8191495549997091, 'mean_cv_val_f1': 0.8184872002780029, 'mean_val2_f1': 0.82996632996633, 'mean_train_auc': 0.9080138651780171, 'mean_cv_val_auc': 0.9088817778145989, 'mean_val2_auc': 0.9095316371259767, 'f1_diff_cv_val': 0.0006623547217061709, 'auc_diff_cv_val': -0.0008679126365818135, 'f1_diff_val2': -0.010816774966620923, 'auc_diff_val2': -0.0015177719479596563, 'params': {'iterations': 100, 'learning_rate': None, 'depth': 3, 'l2_leaf_reg': 1}}, "{'iterations': 100, 'learning_rate': None, 'depth': 3, 'l2_leaf_reg': 3}": {'mean_train_f1': 0.8203011452822511, 'mean_cv_val_f1': 0.8194782608695652, 'mean_val2_f1': 0.8312342569269521, 'mean_train_auc': 0.9080053206333205, 'mean_cv_val_auc': 0.9089152389861771, 'mean_val2_auc': 0.910270154373928, 'f1_diff_cv_val': 0.0008228844126858537, 'auc_diff_cv_val': -0.0009099183528565868, 'f1_diff_val2': -0.01093311164470101, 'auc_diff_val2': -0.0022648

In [19]:
# save results
with open(save_path, 'w') as f:
    json.dump(grid_search_results, f)

In [20]:
# load results
with open(save_path, 'r') as f:
    grid_search_results = json.load(f)

In [21]:
df_res= pd.DataFrame()

for model in grid_search_results:
    temp = grid_search_results[model]
    temp = pd.DataFrame.from_dict(temp)
    temp = temp.transpose()
    temp['model'] = model
    df_res = pd.concat([df_res, temp])

In [22]:
df_res = df_res.reset_index()

In [23]:
top_auc = df_res[(df_res['auc_diff_val2']<=0.05)].sort_values(by='mean_val2_auc', ascending=False).head(500)
top_auc

,index,mean_train_f1,mean_cv_val_f1,mean_val2_f1,mean_train_auc,mean_cv_val_auc,mean_val2_auc,f1_diff_cv_val,auc_diff_cv_val,f1_diff_val2,auc_diff_val2,params,model
1824,"{'n_estimators': 250, 'learning_rate': None, '...",0.999809,0.870499,0.88835,1.0,0.949679,0.956318,0.12931,0.050321,0.111459,0.043682,"{'n_estimators': 250, 'learning_rate': None, '...",lightgbm
1936,"{'n_estimators': 250, 'learning_rate': 0.1, 'm...",0.999809,0.870499,0.88835,1.0,0.949679,0.956318,0.12931,0.050321,0.111459,0.043682,"{'n_estimators': 250, 'learning_rate': 0.1, 'm...",lightgbm
2216,"{'n_estimators': 500, 'learning_rate': 0.1, 'm...",1.0,0.872691,0.877023,1.0,0.951288,0.955382,0.127309,0.048712,0.122977,0.044618,"{'n_estimators': 500, 'learning_rate': 0.1, 'm...",lightgbm
2104,"{'n_estimators': 500, 'learning_rate': None, '...",1.0,0.872691,0.877023,1.0,0.951288,0.955382,0.127309,0.048712,0.122977,0.044618,"{'n_estimators': 500, 'learning_rate': None, '...",lightgbm
2107,"{'n_estimators': 500, 'learning_rate': None, '...",1.0,0.875911,0.880914,1.0,0.952059,0.955305,0.124089,0.047941,0.119086,0.044695,"{'n_estimators': 500, 'learning_rate': None, '...",lightgbm
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,"{'n_estimators': 100, 'learning_rate': None, '...",0.94588,0.866817,0.868357,0.990481,0.946247,0.947637,0.079063,0.044234,0.077524,0.042844,"{'n_estimators': 100, 'learning_rate': None, '...",lightgbm
1707,"{'n_estimators': 100, 'learning_rate': 0.1, 'm...",0.94588,0.866817,0.868357,0.990481,0.946247,0.947637,0.079063,0.044234,0.077524,0.042844,"{'n_estimators': 100, 'learning_rate': 0.1, 'm...",lightgbm
2664,"{'iterations': 100, 'learning_rate': 0.3, 'dep...",0.932777,0.861111,0.873489,0.983856,0.943987,0.947625,0.071666,0.039869,0.059288,0.036231,"{'iterations': 100, 'learning_rate': 0.3, 'dep...",catboost
2037,"{'n_estimators': 250, 'learning_rate': 0.2, 'm...",0.971451,0.861718,0.874494,0.99702,0.945118,0.947618,0.109733,0.051902,0.096957,0.049402,"{'n_estimators': 250, 'learning_rate': 0.2, 'm...",lightgbm


In [24]:
topf1 = df_res[(df_res['f1_diff_val2']<=0.05)].sort_values(by='mean_val2_f1', ascending=False).head(500)
topf1

,index,mean_train_f1,mean_cv_val_f1,mean_val2_f1,mean_train_auc,mean_cv_val_auc,mean_val2_auc,f1_diff_cv_val,auc_diff_cv_val,f1_diff_val2,auc_diff_val2,params,model
2632,"{'iterations': 100, 'learning_rate': 0.2, 'dep...",0.916376,0.865017,0.878049,0.976579,0.946804,0.948842,0.051359,0.029775,0.038327,0.027737,"{'iterations': 100, 'learning_rate': 0.2, 'dep...",catboost
2802,"{'iterations': 250, 'learning_rate': 0.2, 'dep...",0.922117,0.865845,0.876735,0.978939,0.947217,0.949259,0.056272,0.031722,0.045383,0.02968,"{'iterations': 250, 'learning_rate': 0.2, 'dep...",catboost
913,"{'n_estimators': 100, 'learning_rate': 0.1, 'm...",0.922878,0.863764,0.876331,0.98074,0.944757,0.949119,0.059114,0.035983,0.046547,0.031621,"{'n_estimators': 100, 'learning_rate': 0.1, 'm...",xgboost
2628,"{'iterations': 100, 'learning_rate': 0.2, 'dep...",0.897854,0.86768,0.875924,0.966701,0.946108,0.946619,0.030174,0.020593,0.021929,0.020082,"{'iterations': 100, 'learning_rate': 0.2, 'dep...",catboost
676,"{'n_estimators': 50, 'learning_rate': 0.2, 'ma...",0.901298,0.861414,0.874487,0.969478,0.94083,0.944375,0.039884,0.028647,0.026811,0.025102,"{'n_estimators': 50, 'learning_rate': 0.2, 'ma...",xgboost
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1146,"{'n_estimators': 250, 'learning_rate': 0.1, 'm...",0.889083,0.857787,0.860656,0.961657,0.941813,0.943325,0.031296,0.019844,0.028427,0.018332,"{'n_estimators': 250, 'learning_rate': 0.1, 'm...",xgboost
1724,"{'n_estimators': 100, 'learning_rate': 0.2, 'm...",0.883887,0.854855,0.860656,0.958023,0.940533,0.944564,0.029032,0.01749,0.023231,0.01346,"{'n_estimators': 100, 'learning_rate': 0.2, 'm...",lightgbm
1150,"{'n_estimators': 250, 'learning_rate': 0.1, 'm...",0.889534,0.862162,0.860636,0.962337,0.942544,0.945159,0.027372,0.019793,0.028898,0.017177,"{'n_estimators': 250, 'learning_rate': 0.1, 'm...",xgboost
1018,"{'n_estimators': 250, 'learning_rate': None, '...",0.89137,0.860694,0.860616,0.963283,0.942569,0.941779,0.030675,0.020714,0.030754,0.021505,"{'n_estimators': 250, 'learning_rate': None, '...",xgboost


In [25]:
top_both = pd.merge(top_auc, topf1, on=['index'], how='inner')

# keep _x columns
top_both = top_both[[col for col in top_both.columns if '_x' in col]]
top_both.columns = [col.replace('_x', '') for col in top_both.columns]

In [26]:
top_both['sum_diff_val2'] = top_both['f1_diff_val2'] + top_both['auc_diff_val2']
top_both['sum_auc_f1_val2'] = top_both['mean_val2_f1'] + top_both['mean_val2_auc']

top_both = top_both.sort_values(by='sum_auc_f1_val2', ascending=False).head(1000).reset_index(drop=True)
top_both

,mean_train_f1,mean_cv_val_f1,mean_val2_f1,mean_train_auc,mean_cv_val_auc,mean_val2_auc,f1_diff_cv_val,auc_diff_cv_val,f1_diff_val2,auc_diff_val2,params,model,sum_diff_val2,sum_auc_f1_val2
0,0.916376,0.865017,0.878049,0.976579,0.946804,0.948842,0.051359,0.029775,0.038327,0.027737,"{'iterations': 100, 'learning_rate': 0.2, 'dep...",catboost,0.066063,1.826891
1,0.922117,0.865845,0.876735,0.978939,0.947217,0.949259,0.056272,0.031722,0.045383,0.02968,"{'iterations': 250, 'learning_rate': 0.2, 'dep...",catboost,0.075063,1.825994
2,0.91115,0.869878,0.873582,0.973768,0.947791,0.952011,0.041273,0.025977,0.037568,0.021758,"{'iterations': 250, 'learning_rate': 0.1, 'dep...",catboost,0.059326,1.825593
3,0.922878,0.863764,0.876331,0.98074,0.944757,0.949119,0.059114,0.035983,0.046547,0.031621,"{'n_estimators': 100, 'learning_rate': 0.1, 'm...",xgboost,0.078168,1.825449
4,0.911249,0.866891,0.872638,0.974856,0.947641,0.951741,0.044358,0.027215,0.038611,0.023114,"{'iterations': 250, 'learning_rate': 0.1, 'dep...",catboost,0.061725,1.824379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,0.893467,0.864378,0.862295,0.964146,0.946492,0.949159,0.029088,0.017654,0.031171,0.014987,"{'iterations': 250, 'learning_rate': 0.1, 'dep...",catboost,0.046159,1.811454
77,0.910157,0.864919,0.863192,0.973286,0.946118,0.947918,0.045237,0.027167,0.046964,0.025368,"{'iterations': 500, 'learning_rate': 0.2, 'dep...",catboost,0.072332,1.81111
78,0.89707,0.864962,0.862745,0.965643,0.944154,0.947737,0.032108,0.021489,0.034325,0.017906,"{'iterations': 100, 'learning_rate': 0.3, 'dep...",catboost,0.052231,1.810482
79,0.888313,0.864993,0.861157,0.962063,0.945244,0.948706,0.023321,0.016818,0.027156,0.013356,"{'iterations': 500, 'learning_rate': None, 'de...",catboost,0.040513,1.809863


In [27]:
params, model_name = top_both['params'][0], top_both['model'][0]
print('Winner model: |{}|'.format(model_name), 'with params: {}'.format(params))

Winner model: |catboost| with params: {'iterations': 100, 'learning_rate': 0.2, 'depth': 8, 'l2_leaf_reg': 5}


In [28]:
# train model with best params
model = return_model_with_param(model_name, params)
model.fit(X_train_scaled, y_train)

In [29]:
# save model
import pickle
with open('predictive_models/{}.pkl'.format(model_name), 'wb') as f:
    pickle.dump(model, f)


In [30]:
# save scaler
with open('predictive_models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [31]:
# calculate metrics on test set
from sklearn.metrics import roc_auc_score, f1_score

X_test = np.vstack(df[df['sample']=='test']['title_vector'])
y_test = df[df['sample']=='test']['is_clickbait']

X_test_scaled = scaler.transform(X_test)

y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)[:,1]

print('F1 score: {}'.format(f1_score(y_test, y_pred)))
print('AUC score: {}'.format(roc_auc_score(y_test, y_pred_proba)))

F1 score: 0.8602941176470589
AUC score: 0.9433612355639807


In [32]:
# find best threshold for f1 score
import numpy as np
from tqdm.auto import tqdm
# find threshold for best f1 score
thresholds = np.linspace(0, 1, 10000)
f1s = dict()
for threshold in tqdm(thresholds):
    y_pred = (model.predict_proba(X_test_scaled)[:, 1] >= threshold).astype(int)
    f1 = f1_score(y_test, y_pred)
    f1s[threshold] = f1

# get threshold for best f1 score
threshold = max(f1s, key=f1s.get)
threshold

  0%|          | 0/10000 [00:00<?, ?it/s]

0.38823882388238823

In [33]:
# find auc and f1 for best threshold
y_pred = (model.predict_proba(X_test_scaled)[:, 1] >= threshold).astype(int)
y_proba = model.predict_proba(X_test_scaled)[:, 1]

roc_auc = roc_auc_score(y_test, y_proba)
f1 = f1_score(y_test, y_pred)

print(f'ROC AUC test: {roc_auc}')
print(f'F1 test: {f1}')

ROC AUC test: 0.9433612355639807
F1 test: 0.8673114119922631


In [34]:
# find auc and f1 for best threshold on train
y_pred = (model.predict_proba(X_train_scaled)[:, 1] >= threshold).astype(int)
y_proba = model.predict_proba(X_train_scaled)[:, 1]

roc_auc = roc_auc_score(y_train, y_proba)
f1 = f1_score(y_train, y_pred)

print(f'ROC AUC train: {roc_auc}')
print(f'F1 train: {f1}')

ROC AUC train: 0.9734385750508231
F1 train: 0.9096811644056502


In [35]:
# save threshold to txt file
with open('predictive_models/threshold.txt', 'w') as f:
    f.write(str(threshold))

In [39]:
def get_undropped_dimensions(dropped_dimensions, n_dim):
    all_dimensions = list(range(n_dim))
    undropped_dimensions = [dim for dim in all_dimensions if dim not in dropped_dimensions]
    return undropped_dimensions

undropped_dimensions = get_undropped_dimensions(variables_to_drop, 50)
undropped_dimensions = [str('dim_'+str(dim)) for dim in undropped_dimensions]
undropped_dimensions = np.array(undropped_dimensions)
undropped_dimensions

array(['dim_1', 'dim_3', 'dim_4', 'dim_5', 'dim_8', 'dim_12', 'dim_13',
       'dim_14', 'dim_15', 'dim_16', 'dim_18', 'dim_19', 'dim_21',
       'dim_23', 'dim_25', 'dim_26', 'dim_28', 'dim_29', 'dim_35',
       'dim_36', 'dim_37', 'dim_38', 'dim_43', 'dim_44', 'dim_45',
       'dim_47', 'dim_49'], dtype='<U6')

In [45]:
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=undropped_dimensions)

In [47]:
import dalex
# Create an explainer object
exp = dalex.Explainer(model, X_train_scaled_df, y_train, label='Champion Model')

# Calculate feature importance
feature_importance = exp.model_parts()
feature_importance.plot(max_vars=27)


Preparation of a new explainer is initiated

  -> data              : 46618 rows 27 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 46618 values
  -> model_class       : catboost.core.CatBoostClassifier (default)
  -> label             : Champion Model
  -> predict function  : <function yhat_proba_default at 0x2b3df44c0> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.000246, mean = 0.49, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.994, mean = 0.000203, max = 0.996
  -> model_info        : package catboost

A new explainer has been created!


In [48]:
feature_importance

,variable,dropout_loss,label
0,_full_model_,0.026810,Champion Model
1,dim_4,0.029707,Champion Model
2,dim_25,0.030074,Champion Model
3,dim_44,0.030193,Champion Model
4,dim_14,0.030757,Champion Model
5,dim_13,0.030921,Champion Model
6,dim_47,0.031173,Champion Model
7,dim_43,0.031248,Champion Model
8,dim_21,0.031281,Champion Model
9,dim_23,0.031315,Champion Model
